# Simple gene expression model.

This notebook implements the simple gene expression model and provides an example on how to use the software for solving CMEs.

In [1]:
import torch as tn
import torchtt as tntt
import TTCME
import matplotlib.pyplot as plt 
import scipy.integrate
import numpy as np
import datetime

tn.set_default_tensor_type(tn.DoubleTensor)

Define the 4 reactions and create the reaction system object.
Set the state truncation to `[40,120]` and construct the generator in the TT format.

In [2]:
r1 = TTCME.ChemicalReaction(['mRNA','protein'],'mRNA->mRNA+protein', 0.015)
r2 = TTCME.ChemicalReaction(['mRNA','protein'],'mRNA->', 0.002)
r3 = TTCME.ChemicalReaction(['mRNA','protein'],'->mRNA', 0.1)
r4 = TTCME.ChemicalReaction(['mRNA','protein'],'protein->', 0.01)

# Create a reaction system object.
rs = TTCME.ReactionSystem(['mRNA','protein'],[r1, r2, r3, r4])


N = [80,120] 

Att = rs.generatorTT(N)

Define the time integrator.

In [3]:
integrator = TTCME.TimeIntegrator.TTInt(Att, N_max=8, dt_max = 100, method='cheby')

p0 = TTCME.pdf.SingularPMF(N,[2,4])
p = p0.copy()

dt = 16 


Perform the time stepping using the TT solver

In [4]:
time = 0
Ps = [p.dofs.numpy()]
for i in range(64):
    tme = datetime.datetime.now()
    p.dofs = integrator.solve(p.dofs, dt, intervals = 1)
    tme = datetime.datetime.now() - tme
    time += dt
    print('t = %g, solver time %g s'%(time,tme.total_seconds()))
    Ps.append(p.dofs.numpy())



t = 16, solver time 0.914446 s
t = 32, solver time 1.249 s
t = 48, solver time 1.29388 s
t = 64, solver time 1.20559 s
t = 80, solver time 1.15222 s
t = 96, solver time 1.05545 s
t = 112, solver time 1.09409 s
t = 128, solver time 1.05898 s
t = 144, solver time 1.02134 s
t = 160, solver time 1.03353 s
t = 176, solver time 1.00625 s
t = 192, solver time 1.02897 s
t = 208, solver time 1.07473 s
t = 224, solver time 1.03323 s
t = 240, solver time 1.03487 s
t = 256, solver time 0.999669 s
t = 272, solver time 0.997846 s
t = 288, solver time 1.01038 s
t = 304, solver time 0.976749 s
t = 320, solver time 0.989737 s
t = 336, solver time 0.96378 s
t = 352, solver time 0.944488 s
t = 368, solver time 0.928084 s
t = 384, solver time 0.868794 s
t = 400, solver time 0.839485 s
t = 416, solver time 0.653117 s
t = 432, solver time 0.629823 s
t = 448, solver time 0.608421 s
t = 464, solver time 0.561094 s
t = 480, solver time 0.533945 s
t = 496, solver time 0.490571 s
t = 512, solver time 0.616635 s


Compute the solution without the TT solver.

In [ ]:
Asp = rs.generator_sparse(N)

res = scipy.integrate.solve_ivp(lambda t,y: Asp.dot(y), [0,time],p0.dofs.numpy().flatten(),max_step=time/10000)
Pt = res.y.reshape(N+[-1])
p_ref = Pt[:,:,-1]

Plot the TT solution, the solution obtained with the classical ODE solver and the error between them.

In [ ]:
plt.figure()
plt.imshow(p.dofs.full(), origin='lower',cmap='gray_r')
plt.colorbar()
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')

plt.figure()
plt.imshow(p_ref, origin='lower',cmap='gray_r')
plt.colorbar()
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')

plt.figure()
plt.imshow(np.abs(p.dofs.numpy()-p_ref), origin='lower',cmap='gray_r')
plt.colorbar()
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')